In [ ]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


In [ ]:
classifier = Sequential()
classifier.add(Convolution2D(32, 3, 3, input_shape= (64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Add a second convolutional layer, and you don't need an input shape parameter because Keras already knows from the previous set
classifier.add(Convolution2D(32, 3, 3, activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(output_dim= 128, activation='relu'))
classifier.add(Dense(output_dim= 1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
cwd = '/home/devinlax/jupyter/Artificial_Neural_Networks/Convolutional_Neural_Networks/dataset/'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
training_set = train_datagen.flow_from_directory(
        cwd + 'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


In [ ]:

test_set = test_datagen.flow_from_directory(
        cwd + 'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


In [ ]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000,
        epochs=1,
        validation_data=test_set,
        validation_steps=2000)


In [ ]:
classifier.

